In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import json

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.bbpredict
import pymedphys._wlutz.pylinac
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

In [ ]:
working_dir = pathlib.Path(r"\\pdc\OneDrive$\RCCC Specific Files\Linac Beam Data Record\Synergy 2694\QA\20200807_unstable_6FFF_investigation\wlutz")
output_dir = working_dir.joinpath('results')
output_dir.mkdir(exist_ok=True)

In [ ]:
cache_path = working_dir.joinpath("cache.json")

In [ ]:
penumbra = 2
edge_lengths = [20, 24]
initial_rotation = 0
bb_diameter = 8

bb_predictor_tol = 0.2

pd.set_option("display.max_rows", 101)

In [ ]:
clockwise_string = "00_CW"
counter_clockwise_string = "01_CC"

In [ ]:
directions_map = {
    clockwise_string: "clockwise",
    counter_clockwise_string: "counter-clockwise"
}

In [ ]:
frame_paths_list = list(working_dir.joinpath("frames").glob("**/*.tif"))
# frame_paths_list

In [ ]:
frame = [path.stem.split('_')[1] for path in frame_paths_list]
timestamps = [path.parent.stem for path in frame_paths_list]
directions = [directions_map[path.parent.parent.stem] for path in frame_paths_list]
beams = [path.parent.parent.parent.stem for path in frame_paths_list]

keys = list(zip(beams, directions, timestamps, frame))

image_paths = {
    key: path for key, path in zip(keys, frame_paths_list)
}

In [ ]:
# image_paths

In [ ]:
key_map = {
    key: '-'.join(key) for key in keys
}

inv_key_map = {
    item: key for key, item in key_map.items()
}

In [ ]:
movie_keys = list({
    key[0:3] for key in keys
})

In [ ]:
movie_output_dirs = {}

for key in movie_keys:
    movie_output_dirs[key] = output_dir.joinpath(f"{key[0]} {key[1]} {key[2]}")
    movie_output_dirs[key].mkdir(exist_ok=True)
    movie_output_dirs[key].joinpath('images').mkdir(exist_ok=True)

In [ ]:
data = {}

In [ ]:
try:
    with open(cache_path, 'r') as a_file:
        data_string_keys = json.load(a_file)
    
    data = {
        inv_key_map[key]: item for key, item in data_string_keys.items()
    }
except FileNotFoundError:
    data = {}

In [ ]:
def plot_pylinac_comparison(field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation, pylinac):
    try:
        bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
            field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation, pylinac_tol=np.inf
        )

        fig = pymedphys._wlutz.reporting.image_analysis_figure(
            x,
            y,
            img,
            bb_centre,
            field_centre,
            field_rotation,
            bb_diameter,
            edge_lengths,
            penumbra,
        )

        plt.title('PyMedPhys Basinhopping Method')
    except Exception as e:
        print(e)

    try:
        fig = pymedphys._wlutz.reporting.image_analysis_figure(
            x,
            y,
            img,
            pylinac['v2.2.6']['bb_centre'],
            pylinac['v2.2.6']['field_centre'],
            field_rotation,
            bb_diameter,
            edge_lengths,
            penumbra,
        )

        plt.title('Pylinac v2.2.6 Filter and Profile Method')
    except Exception as e:
        print(e)
        
    try:
        fig = pymedphys._wlutz.reporting.image_analysis_figure(
            x,
            y,
            img,
            pylinac['v2.2.7']['bb_centre'],
            pylinac['v2.2.7']['field_centre'],
            field_rotation,
            bb_diameter,
            edge_lengths,
            penumbra,
        )

        plt.title('Pylinac v2.2.7 Filter and Scikit-Image Method')
    except Exception as e:
        print(e)
    
    
    plt.show()

In [ ]:
for key, image_path in image_paths.items():
    try:
        this_data = data[key]
        pymedphys_data = this_data['pymedphys']
    except KeyError:
        this_data = {}
        pymedphys_data = {}
        this_data['pymedphys'] = pymedphys_data
        data[key] = this_data
    
    try:
        pymedphys_data['field_centre']
        pymedphys_data['field_rotation']
        this_data['pylinac']
        pymedphys_data['bb_centre']
    except KeyError:
        print(key)
        x, y, img = pymedphys._wlutz.iview.iview_image_transform_from_path(image_path)

        field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
        initial_centre = pymedphys._wlutz.findfield.get_centre_of_mass(x, y, img)
    
    try:
        pymedphys_data['field_centre']
        pymedphys_data['field_rotation']
    except KeyError:
    
        try:
            pymedphys_data['field_centre'], pymedphys_data['field_rotation'] = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
                field, edge_lengths, penumbra, initial_centre
            )            
        except ValueError as e:
            print(e)
            continue  
        
        pymedphys_data['field_centre'] = pymedphys_data['field_centre']
        pymedphys_data['field_rotation'] = pymedphys_data['field_rotation']
    
    try:
        this_data['pylinac']
    except KeyError:
        try:
            this_data['pylinac'] = pymedphys._wlutz.pylinac.run_wlutz(
                field, edge_lengths, penumbra, pymedphys_data['field_centre'], pymedphys_data['field_rotation'])
        except Exception as e:
            print(e)
            pass
    
    
    try:
        pymedphys_data['bb_centre']
    except KeyError:
        try:
            pymedphys_data['bb_centre'] = pymedphys._wlutz.findbb.optimise_bb_centre(
                field, bb_diameter, edge_lengths, penumbra, pymedphys_data['field_centre'], pymedphys_data['field_rotation']
            )
        except pymedphys._wlutz.pylinac.PylinacComparisonDeviation as e:
            print(e)
            
            try:
                plot_pylinac_comparison(
                    field, bb_diameter, edge_lengths, penumbra, pymedphys_data['field_centre'], pymedphys_data['field_rotation'],
                    this_data['pylinac']
                )
            except ValueError as e:
                print(e)
                continue

            continue
        except ValueError as e:
            print(e)
            continue
    
        pymedphys_data['bb_centre'] = pymedphys_data['bb_centre']

In [ ]:
data_for_json = {
    key_map[key]: item for key, item in data.items()
}

with open(cache_path, 'w') as a_file:
    json.dump(data_for_json, a_file, indent=2)

In [ ]:
# data.keys()

In [ ]:
# key_map

In [ ]:

movie_data_dicts = {
    movie_key: {
        int(key[3]): item for key, item in data.items()
        if key[0:3] == movie_key
    }
    for movie_key in movie_keys
}

In [ ]:
for key, item in movie_data_dicts.items():
    assert list(sorted(item.keys())) == list(range(len(item.keys())))

In [ ]:
movie_data = {
    movie_key: [item[frame_key] for frame_key in sorted(item.keys())]
    for movie_key, item in movie_data_dicts.items()
}

In [ ]:
def extract_data(keys, data, lookup_func):
    result = {}

    for key in keys:
        result[key] = []
        for item in data[key]:
            try:
                result[key].append(lookup_func(item))
            except KeyError:
                result[key].append(np.nan)

        result[key] = np.array(result[key])
        
    return result

In [ ]:
pymedphys_field_rotations = extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_rotation'])

In [ ]:
def check_for_changed_not_nan(diff, not_nan):
    unreasonable = np.abs(diff) > 60
    new_not_nan = np.copy(not_nan)
    new_not_nan[new_not_nan] = np.invert(unreasonable)

    diff = diff[np.invert(unreasonable)]
    
    return new_not_nan

In [ ]:
def calc_gantry(rotation, not_nan, direction_key):
    nan_removed_rotation = rotation[not_nan]
    
    if direction_key == 'clockwise':
        diff = np.diff(np.concatenate([[-180], nan_removed_rotation]))
        diff[diff > 0] = diff[diff > 0] - 180
               
        new_not_nan = check_for_changed_not_nan(diff, not_nan)
        if np.any(new_not_nan != not_nan):
            gantry, not_nan = calc_gantry(rotation, new_not_nan, direction_key)
        else:
            gantry = -180 - np.cumsum(diff * 2)
    elif direction_key == 'counter-clockwise':
        diff = np.diff(np.concatenate([[0], nan_removed_rotation]))
        diff[diff < 0] = diff[diff < 0] + 180
        
        new_not_nan = check_for_changed_not_nan(diff, not_nan)
        if np.any(new_not_nan != not_nan):
            gantry, not_nan = calc_gantry(rotation, new_not_nan, direction_key)
        else:
            gantry = 180 - np.cumsum(diff * 2)
    else:
        raise ValueError("Expected one of 'clockwise' or 'counter-clockwise'")
        
    return gantry, not_nan
        
    

In [ ]:
def determine_gantry_angle(direction_key, rotation):
    not_nan = np.invert(np.isnan(rotation))
    gantry, not_nan = calc_gantry(rotation, not_nan, direction_key)
        
    gantry_with_nans = np.ones_like(rotation) * np.nan
    out_of_bounds = np.logical_or(gantry < -180, gantry > 180)
    gantry[out_of_bounds] = np.nan
    gantry_with_nans[not_nan] = gantry
        
    return gantry_with_nans

In [ ]:
gantry_angles = {}

for key in movie_keys:
    direction_key = key[1]
    rotation = pymedphys_field_rotations[key]

    if len(rotation) != 0:
        gantry_angles[key] = determine_gantry_angle(direction_key, rotation)

In [ ]:
len(gantry_angles.keys())

movie_keys = list(gantry_angles.keys())

In [ ]:
movie_keys

In [ ]:
# pymedphys_field_rotations

In [ ]:
columns=[
    'Image Frame', 'Gantry Angle (deg)', 'Field x (mm)', 'Field y (mm)', 'BB x (mm)', 'BB y (mm)'
]

In [ ]:
# prep_for_dataframe

In [ ]:
movie_keys

In [ ]:
len(movie_keys)

In [ ]:
prep_for_dataframe = [
    gantry_angles,
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_centre'][1]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['bb_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['bb_centre'][1]),
]

len(prep_for_dataframe[3].keys())

In [ ]:




dataframes = {}



for key in movie_keys:
    print(key)
    prepped_data = [item[key] for item in prep_for_dataframe]
    frames = [list(range(len(prepped_data[0])))]
    
    dataframe_data = np.vstack(frames + prepped_data).T
    
    dataframe = pd.DataFrame(
        columns=columns,
        data=dataframe_data
    )
    
    dataframe['Image Frame'] = dataframe['Image Frame'].astype(np.int64)
    dataframe = dataframe.set_index('Image Frame')
    
    dataframes[key] = dataframe

In [ ]:
# dataframes[key]

In [ ]:
bb_x_predictor_data = [
    dataframes[key]['BB x (mm)'] for key in movie_keys
]
bb_y_predictor_data = [
    dataframes[key]['BB y (mm)'] for key in movie_keys
]
gantry_predictor_data = [
    gantry_angles[key] for key in movie_keys
]
direction_predictor_data = [key[1] for key in movie_keys]

predict_bb = pymedphys._wlutz.bbpredict.create_bb_predictor(
    bb_x_predictor_data, bb_y_predictor_data, gantry_predictor_data, direction_predictor_data, default_tol=bb_predictor_tol)

predict_bb([0, 2], 'clockwise')

In [ ]:
gantry_i = np.linspace(-180, 180, 401)

In [ ]:
plt.figure(figsize=(12,10))

for g, x, key in zip(gantry_predictor_data, bb_x_predictor_data, movie_keys):
    if key[1] == 'clockwise':
        prop = '-'
    else:
        prop = '--'
        
    plt.plot(g, x, prop, alpha=0.5, label=key[0:2])
    
plt.plot(gantry_i, predict_bb(gantry_i, 'clockwise')[0], 'k')
plt.plot(gantry_i, predict_bb(gantry_i, 'counter-clockwise')[0], 'k--')
plt.legend()

plt.title("Absolute BB iView x position predictor")
plt.xlabel("Gantry Angle (deg)")
plt.ylabel("iView absolute x-position (mm)")

plt.savefig(output_dir.joinpath("Absolute BB x position predictor.png"))

In [ ]:
plt.figure(figsize=(12,10))

for g, y, key in zip(gantry_predictor_data, bb_y_predictor_data, movie_keys):
    if key[1] == 'clockwise':
        prop = '-'
    else:
        prop = '--'
        
    plt.plot(g, y, prop, alpha=0.5, label=key[0:2])
    
plt.plot(gantry_i, predict_bb(gantry_i, 'clockwise')[1], 'k')
plt.plot(gantry_i, predict_bb(gantry_i, 'counter-clockwise')[1], 'k--')
plt.legend()

plt.title("Absolute BB iView y position predictor")
plt.xlabel("Gantry Angle (deg)")
plt.ylabel("iView absolute y-position (mm)")

plt.savefig(output_dir.joinpath("Absolute BB y position predictor.png"))

In [ ]:
for key in movie_keys:
    bb_x = dataframes[key]['BB x (mm)'].copy()
    bb_y = dataframes[key]['BB y (mm)'].copy()
    gantry = dataframes[key]['Gantry Angle (deg)']
    direction = key[1]
    
    isnan = np.isnan(bb_x)
    assert np.all(isnan == np.isnan(bb_y))
    
    bb_x_prediction, bb_y_prediction = predict_bb(gantry[isnan], direction)
    
    bb_x[isnan] = bb_x_prediction
    bb_y[isnan] = bb_y_prediction
    
    dataframes[key]['BB x [with predictions] (mm)'] = bb_x
    dataframes[key]['BB y [with predictions] (mm)'] = bb_y

In [ ]:
pylinac_columns = [
    'Pylinac Field x (mm)', 'Pylinac Field y (mm)',
    'Pylinac v2.2.6 BB x (mm)', 'Pylinac v2.2.6 BB y (mm)',
    'Pylinac v2.2.7 BB x (mm)', 'Pylinac v2.2.7 BB y (mm)'
]


pylinac_data_extract = [
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.7']['field_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.7']['field_centre'][1]),
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.6']['bb_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.6']['bb_centre'][1]),
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.7']['bb_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pylinac']['v2.2.7']['bb_centre'][1]),
]

for key in movie_keys:
    for column, pylinac_data in zip(pylinac_columns, pylinac_data_extract):
        dataframes[key][column] = pylinac_data[key]

In [ ]:
for key in movie_keys:   
    dataframes[key]['Field - BB x (mm)'] = dataframes[key]['Field x (mm)'] - dataframes[key]['BB x (mm)']
    dataframes[key]['Field - BB y (mm)'] = dataframes[key]['Field y (mm)'] - dataframes[key]['BB y (mm)']

In [ ]:
def plot_enery_axis(energy, axis, dataframes):
    plt.figure(figsize=(12,10))

    for key in movie_keys:
        if energy in key[0]:
            if key[1] == 'clockwise':
                prop = '-'
            else:
                prop = '--'

            plt.plot(
                dataframes[key]['Gantry Angle (deg)'], 
                dataframes[key][f'Field - BB {axis} (mm)'], 
                prop, label=key[0:3], alpha=0.8)
            
    x = np.linspace(-180, 180)

#     if axis == 'y':
#         plt.plot(x, 0.6*np.cos(x*np.pi/180), 'k', label='"Ideal"')
#         plt.plot(x, 0.6*np.cos(x*np.pi/180)-0.5, 'r', label='0.5 mm "bounds"', alpha=0.2)
#         plt.plot(x, 0.6*np.cos(x*np.pi/180)+0.5, 'r', alpha=0.2)
        
#     elif axis == 'x':
#         plt.plot(x, np.zeros_like(x), 'k', label='"Ideal"')
#         plt.plot(x, np.zeros_like(x)-0.5, 'r', label='0.5 mm "bounds"', alpha=0.2)
#         plt.plot(x, np.zeros_like(x)+0.5, 'r', alpha=0.2)

    plt.grid(True)
#     plt.ylim([-2,2])

    
    plt.legend()
    plt.title(f"{energy} | iView panel {axis}-axis")
    plt.xlabel('Gantry (deg)')
    plt.ylabel(f'Field centre {axis} - BB centre {axis} (mm)')
    

In [ ]:
energies = ['06MV', '10MV', '06FFF']
axes = ['x', 'y']

dose_rate_ranges = {
    "06MV": {
        "low": (0, 200),
        "half": (200, 400),
        "full": (400, 700)
    },
    "10MV": {
        "low": (0, 200),
        "half": (200, 400),
        "full": (400, 700)
    },
    "06FFF": {
        "low": (0, 500),
        "half": (500, 1100),
        "full": (1100, 2000)
    },
}

In [ ]:
def get_energy_and_dose_rate_label_from_key(key):
    split_key = key[0].split('_')
    energy = split_key[1]
    
    dose_rate = int(split_key[-1].replace('DR', ''))
    ranges = dose_rate_ranges[energy]
    
    dose_rate_label = None
    for label, dose_rates in ranges.items():
        if dose_rate >= dose_rates[0] and dose_rate < dose_rates[1]:
            dose_rate_label = label
            break
            
    return energy, dose_rate_label

In [ ]:
df_key_to_stat_key = {
    key: get_energy_and_dose_rate_label_from_key(key)
    for key in dataframes.keys()
}

df_keys = df_key_to_stat_key.keys()

# energy_and_dose_rate_labels

In [ ]:
df_key_to_stat_key.values()

In [ ]:
stat_key_to_df_key = {}

for stat_key in set(df_key_to_stat_key.values()):    
    stat_key_to_df_key[stat_key] = [
        df_key for df_key in df_keys if df_key_to_stat_key[df_key] == stat_key
    ]
    
# stat_key_to_df_key

In [ ]:
statistics = {}

for stat_key, df_keys in stat_key_to_df_key.items():
    x_panel_data = []
    y_panel_data = []
    
    for df_key in df_keys:
        x_panel_data += dataframes[df_key]['Field - BB x (mm)'].values.tolist()
        y_panel_data += dataframes[df_key]['Field - BB y (mm)'].values.tolist()
        
    statistics[stat_key] = {
        "x": {
            "min": np.nanmin(x_panel_data).round(2),
            "mean": np.nanmean(x_panel_data).round(2),
            "max": np.nanmax(x_panel_data).round(2),
        },
        "y": {
            "min": np.nanmin(y_panel_data).round(2),
            "mean": np.nanmean(y_panel_data).round(2),
            "max": np.nanmax(y_panel_data).round(2),
        }
    }

In [ ]:
# del statistics[('06FFF', 'low')]

In [ ]:
statistics

In [ ]:
stat_columns = [
    'Full DR | min', 'Full DR | mean', 'Full DR | max', 
    'Half DR | min', 'Half DR | mean', 'Half DR | max', 
    'Low DR | min', 'Low DR | mean', 'Low DR | max'
]

stat_index = [
    '06MV | x', '06MV | y',
    '10MV | x', '10MV | y',
    '06FFF | x', '06FFF | y',
]

In [ ]:
table = []

for beam in energies:
    for axis in ['x', 'y']:
        current_row = []
        for label in ['full', 'half', 'low']:
            for stat in ['min', 'mean', 'max']:
                try:
                    current_row.append(statistics[(beam, label)][axis][stat])
                except KeyError:
                    current_row.append(np.nan)
        table.append(current_row)
        
table

In [ ]:
statistics_table = pd.DataFrame(data=table, columns=stat_columns, index=stat_index)

In [ ]:
statistics_table

In [ ]:
# Can select and copy the following table and use "paste special > text" to put into Excel

statistics_table.index = ['x (mm)', 'y (mm)']*3
statistics_table

In [ ]:
for energy in energies:
    for axis in axes:
        plot_enery_axis(energy, axis, dataframes)
        
        plt.savefig(output_dir.joinpath(f"{energy}_{axis}-axis.png"))

In [ ]:
for key in movie_keys:
    print(key)
    IPython.display.display(dataframes[key])
    
    dataframes[key].round(2).to_csv(movie_output_dirs[key].joinpath('raw_results.csv'))

In [ ]:
# RENABLE THE FOLLOWING TO SEE ALL FAILURES

In [ ]:
# try:
#     with open('session_cache.json', 'r') as a_file:
#         data_string_keys = json.load(a_file)
    
#     data = {
#         inv_key_map[key]: item for key, item in data_string_keys.items()
#     }
# except FileNotFoundError:
#     data = {}

In [ ]:
# for key, image_path in image_paths.items():   
#     images_dir = movie_output_dirs[key[0:3]].joinpath('images')
    
#     try:
#         this_data = data[key]
#         pymedphys_data = this_data['pymedphys']
#     except KeyError:
#         continue
        
#     x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)
    
#     try:
#         pymedphys_data['bb_centre']
#         continue
#     except KeyError:
#         pass
    
    
#     try:
#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             None,
#             pymedphys_data['field_centre'],
#             pymedphys_data['field_rotation'],
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         plt.title('PyMedPhys Basinhopping Method')
#         plt.tight_layout()
#         filepath = images_dir.joinpath(f"frame_{key[3]}_PyMedPhys_field_only.png")
#         plt.savefig(filepath)
#         print(f"Saved {filepath}")
#         plt.close()
#     except KeyError:
#         pass

In [ ]:
# RENABLE BELOW FOR DIAGNOSTIC IMAGE SAVING

In [ ]:
# for key, image_path in image_paths.items():
#     print(key)
    
#     images_dir = movie_output_dirs[key[0:3]].joinpath('images')
    
#     try:
#         this_data = data[key]
#         pymedphys_data = this_data['pymedphys']
#     except KeyError:
#         continue
        
#     x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)

#     try:
#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             pymedphys_data['bb_centre'],
#             pymedphys_data['field_centre'],
#             pymedphys_data['field_rotation'],
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         plt.title('PyMedPhys Basinhopping Method')
#         plt.tight_layout()
#         plt.savefig(images_dir.joinpath(f"frame_{key[3]}_PyMedPhys.png"))
#         plt.close()
#     except KeyError:
#         pass
    
#     try:
#         pylinac = this_data['pylinac']
#     except KeyError:
#         continue
    

#     fig = pymedphys._wlutz.reporting.image_analysis_figure(
#         x,
#         y,
#         img,
#         pylinac['v2.2.6']['bb_centre'],
#         pylinac['v2.2.6']['field_centre'],
#         pymedphys_data['field_rotation'],
#         bb_diameter,
#         edge_lengths,
#         penumbra,
#     )

#     plt.title('Pylinac v2.2.6 Filter and Profile Method')
#     plt.tight_layout()
#     plt.savefig(images_dir.joinpath(f"frame_{key[3]}_Pylinac_v2.2.6.png"))
#     plt.close()



#     fig = pymedphys._wlutz.reporting.image_analysis_figure(
#         x,
#         y,
#         img,
#         pylinac['v2.2.7']['bb_centre'],
#         pylinac['v2.2.7']['field_centre'],
#         pymedphys_data['field_rotation'],
#         bb_diameter,
#         edge_lengths,
#         penumbra,
#     )

#     plt.title('Pylinac v2.2.7 Filter and Scikit-Image Method')
#     plt.tight_layout()
#     plt.savefig(images_dir.joinpath(f"frame_{key[3]}_Pylinac_v2.2.7.png"))
#     plt.close()

    